<a href="https://colab.research.google.com/github/mariamamgad8/HumanAI_GSoC_Proposal_2025/blob/main/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **geocoding posts and generating a heatmap of crisis discussions**

In [6]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
import re
import time

# Load your CSV file (replace path if needed)
df = pd.read_csv("/content/classified_tweets.csv")  # Change path as needed

# Basic regex to extract city/place names
def extract_location_simple(text):
    pattern = r"\b(?:in|at|from|near|around)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)?)"
    match = re.search(pattern, str(text))
    return match.group(1) if match else None

df["location_mention"] = df["cleaned_text"].apply(extract_location_simple)
location_df = df.dropna(subset=["location_mention"]).copy()

# Geocode place names using geopy
geolocator = Nominatim(user_agent="crisis_locator", timeout=10)  # Added timeout

def geocode_location(location):
    try:
        loc = geolocator.geocode(location)
        time.sleep(1)  # Respect rate limits
        if loc:
            return (loc.latitude, loc.longitude)  # Return as a tuple
        else:
            return (None, None)  # Explicit return if no result
    except Exception as e:
        print(f"Error geocoding '{location}': {e}")
        return (None, None)  # Ensure always returns a tuple

# Step 1: Geocode and store results in a list
geocoded_results = location_df["location_mention"].apply(geocode_location).tolist()

# Step 2: Assign to DataFrame
location_df["latitude"] = [lat for (lat, lon) in geocoded_results]
location_df["longitude"] = [lon for (lat, lon) in geocoded_results]

# Create a heatmap only if we have valid locations
if not location_df.empty:
    heatmap_map = folium.Map(location=[20, 0], zoom_start=2)
    heat_data = location_df[["latitude", "longitude"]].values.tolist()
    HeatMap(heat_data).add_to(heatmap_map)

    # Save and show results
    heatmap_map.save("crisis_heatmap_fallback.html")
    top_locations = location_df["location_mention"].value_counts().head(5)
    top_locations.to_csv("top_5_locations_fallback.csv", header=["count"])

    print("Heatmap and top 5 locations saved!")
else:
    print(" No valid geocoded locations found to create heatmap.")

 No valid geocoded locations found to create heatmap.
